In [1]:
import pandas as pd
from collections import Counter

df = pd.read_csv('situacao_do_cliente.csv')

X_df = df[['recencia','frequencia','semanas_de_inscricao']]
Y_df = df['situacao']
Xdummies_df = pd.get_dummies(X_df)
Ydummies_df = Y_df

# valores dentro dos dataframes
X = Xdummies_df.values
Y = Ydummies_df.values

porcentagem_treino = 0.8
porcentagem_teste = 0.1
tamanho_treino = int(porcentagem_treino * len(Y))
tamanho_teste = int(porcentagem_teste * len(Y))
tamanho_validacao = len(Y) - tamanho_treino - tamanho_teste

treino_dados = X[:tamanho_treino]
treino_marcacoes = Y[:tamanho_treino]

fim_treino = tamanho_treino + tamanho_teste

teste_dados = X[tamanho_treino:fim_treino]
teste_marcacoes = Y[tamanho_treino:fim_treino]

validacao_dados = X[fim_treino:]
validacao_marcacoes = Y[fim_treino:]

In [2]:
def fit_and_predict(nome, modelo, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes):
    modelo.fit(treino_dados, treino_marcacoes)
    
    resultado = modelo.predict(teste_dados)
    acertos = resultado == teste_marcacoes
    
    # taxa de acertos do algoritmo treinado
    total_acertos = sum(acertos)
    total_elementos = len(teste_marcacoes)
    taxa_acertos = 100.0 * total_acertos / total_elementos
    
    msg = "Taxa de acerto do algoritmo {0}: {1}".format(nome, taxa_acertos)
    print(msg)
    
    print(resultado)
    
    return taxa_acertos

In [3]:
def teste_real(modelo, validacao_dados, validacao_marcacoes):
    modelo.fit(validacao_dados, validacao_marcacoes)
    resultado = modelo.predict(validacao_dados)
    
    acertos = resultado == validacao_marcacoes
    total_acertos = sum(acertos)
    total_elementos = len(validacao_marcacoes)
    
    taxa_acertos = 100.0 * total_acertos / total_elementos
    
    msg = "Taxa de acerto do vencedor entre os dois algoritmos no mundo real: {0}".format(taxa_acertos)
    print(msg)

In [17]:
resultados = {}

# treinando com OnevsRest
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
modeloOneVsRest = OneVsRestClassifier(LinearSVC(random_state = 0))
resultadoOneVsRest = fit_and_predict("OneVsRestClassifier", modeloOneVsRest, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)
resultados[resultadoOneVsRest] = modeloOneVsRest

# treinando com OnevsOne
from sklearn.multiclass import OneVsOneClassifier
modeloOneVsOne = OneVsOneClassifier(LinearSVC(random_state = 0))
resultadoOneVsOne = fit_and_predict("OneVsOneClassifier", modeloOneVsOne, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)
resultados[resultadoOneVsOne] = modeloOneVsOne

# treinando com MultinomialNB
from sklearn.naive_bayes import MultinomialNB
modeloMultinomial = MultinomialNB()
resultadoMultinomial = fit_and_predict("MultinomialNB", modeloMultinomial, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)
resultados[resultadoMultinomial] = modeloMultinomial

# treinando com AdaBoost
from sklearn.ensemble import AdaBoostClassifier
modeloAdaBoost = AdaBoostClassifier()
resultadoAdaBoost = fit_and_predict("AdaBoostClassifier", modeloAdaBoost, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)
resultados[resultadoAdaBoost] = modeloAdaBoost

maximo = max(resultados)
vencedor = resultados[maximo]
    
teste_real(vencedor, validacao_dados, validacao_marcacoes)

# taxa de acertos do algoritmo base
acerto_base = max(Counter(validacao_marcacoes).values())
taxa_acerto_base = 100.0 * acerto_base / len(validacao_marcacoes)
print('Taxa de acerto do algoritmo base: %f' % taxa_acerto_base)
print()
total_elementos = len(validacao_dados)
print('Total de elementos: %d' % total_elementos)

Taxa de acerto do algoritmo OneVsRestClassifier: 90.9090909090909
[2 0 1 1 1 1 1 1 2 1 2 1 1 1 1 1 1 1 1 1 2 1]
Taxa de acerto do algoritmo OneVsOneClassifier: 100.0
[2 0 1 1 1 1 1 1 2 1 2 1 1 0 1 1 1 1 0 1 2 1]
Taxa de acerto do algoritmo MultinomialNB: 72.72727272727273
[1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1]
Taxa de acerto do algoritmo AdaBoostClassifier: 68.18181818181819
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Taxa de acerto do vencedor entre os dois algoritmos no mundo real: 95.65217391304348
Taxa de acerto do algoritmo base: 82.608696

Total de elementos: 23


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
